# Peer-Graded

## One notebook for all three parts.

## Part 1:  transform the data in the table on the Wikipedia page into dataframe

#### Import request library and scrape from wikipedia url.

In [1]:
import requests

In [2]:
web_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Import BeautifulSoup.

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(web_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

#### When inspect the HTML, we can find that info that we are looking for is under class wikitable sortable.

In [4]:
T_table = soup.find('table',{'class':'wikitable sortable'})
T_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [5]:
print(T_table.tr.text)


Postcode
Borough
Neighbourhood



In [6]:
headers = "PostalCode, Borough, Neighborhood"

#### Getting values.

In [7]:
table1=""
for tr in T_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M

In [8]:
file=open("toronto01.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8738

#### Convert into pandas dataframe.

In [9]:
import pandas as pd
df = pd.read_csv('toronto01.csv',header=None)
df.columns=["Postalcode","Borough","Neighborhood"]

In [10]:
df.head(5)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Process only the cells that have an assigned borough. Drop rows with "Not Assigned" borough.

In [11]:
# Only process the cells that have an assigned borough
indexNames = df[ df['Borough'] =='Not assigned'].index

# drop these rows
df.drop(indexNames , inplace=True)

In [12]:
df.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [13]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
df.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### rows with same postcode will be combined into one row.

In [14]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [15]:
df_new=result.reset_index()
df_new.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Print the number of rows of your dataframe

In [16]:
df_new.shape

(103, 3)

## Part 2:  get the latitude and the longitude coordinates of each neighborhood

#### Downloaded the csv file that has the geographical coordinates of each postal code from http://cocl.us/Geospatial_data.  

In [17]:
df_lolat = pd.read_csv('Geospatial_Coordinates.csv')
df_lolat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_lolat.columns=['Postalcode','Latitude','Longitude']

In [19]:
df_lolat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
Toronto_df = pd.merge(df_new,
                 df_lolat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
Toronto_df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Part 3:  Explore and cluster the neighborhoods

#### Get latitude and longitude values of New York.

In [21]:
# import libraries
from geopy.geocoders import Nominatim
import matplotlib.colors as colors
import matplotlib.cm as cm
from sklearn.cluster import KMeans
import folium

In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [23]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare Account and credentials.

In [24]:
CLIENT_ID = '2MBLWPCF03MPNX50MJBLOD1LIJZRYU3NIAWB4GUVCMX3FPQ1' # your Foursquare ID
CLIENT_SECRET = 'UEMUQIT4POBRNMAZ4Y1Y1N3EYBIM12D1MCBY5RBNWBY3PY34' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2MBLWPCF03MPNX50MJBLOD1LIJZRYU3NIAWB4GUVCMX3FPQ1
CLIENT_SECRET:UEMUQIT4POBRNMAZ4Y1Y1N3EYBIM12D1MCBY5RBNWBY3PY34


#### define radius & limit

In [25]:
radius=500
LIMIT=100

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

#### Check dataframe's size.

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(2268, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### How many values returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,57,57,57,57,57,57


#### Analyze each neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### size of new data frame

In [31]:
toronto_onehot.shape

(2268, 276)

#### Group rows by neighborhoods. Take the mean of the frequency.

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.0,0.010000,0.000000,0.00,0.000000,0.0000,0.010000,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.05,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.017544,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


#### top 5 most common venues.

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place   0.2
1        Skating Rink   0.2
2      Breakfast Spot   0.2
3  Chinese Restaurant   0.2
4              Lounge   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.17
1         Grocery Store  0.17
2  Fast Food Restaurant  0.08
3            Beer Store  0.08
4   Fried Chicken Joint  0.08


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Gym  

4              Motel   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.15
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3     Electronics Store  0.05
4            Restaurant  0.05


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2   Restaurant  0.04
3   Steakhouse  0.04
4        Hotel  0.04


----Flemingdon Park, Don Mills South----
                   venue  freq
0                    Gym  0.09
1             Beer Store  0.09
2            Coffee Shop  0.09
3       Asian Restaurant  0.09
4  General Entertainment  0.05


----Forest Hill North, Forest Hill West----
               venue  freq
0      Jewelry Store  0.25
1               Park  0.25
2              Trail  0.25
3   Sushi Restaurant  0.25
4  Accessories Store  0.00


----Glencairn----
                 venue  freq
0  Japanese Restaurant  0.25
1     Sushi Restaurant  0.25
2                  Pub  0.25
3     

                  venue  freq
0           Coffee Shop  0.11
1                  Café  0.04
2            Restaurant  0.04
3                 Hotel  0.03
4  Fast Food Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----The Annex, North Midtown, Yorkville----
            venue  freq
0            Café  0.12
1  Sandwich Place  0.12
2     Coffee Shop  0.12
3     Pizza Place  0.08
4  History Museum  0.04


----The Beaches----
                  venue  freq
0     Health Food Store   0.2
1                   Pub   0.2
2                 Trail   0.2
3  Other Great Outdoors   0.2
4     Accessories Store   0.0


----The Beaches West, India Bazaar----
                  venue  freq
0        Sandwich Place  0.10
1                  Park  0.10
2           Pizza Place  0.10
3          Burger Joint  0.05
4  Fast Food Restaurant  0.05


----T

#### Put into dataframe. Sort in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Display top 10 venues.

In [35]:
import numpy as np

In [36]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [37]:
# create data frame.
neighborhoods_sorted = pd.DataFrame(columns=columns)
neighborhoods_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,American Restaurant,Hotel,Burger Joint,Restaurant
1,Agincourt,Sandwich Place,Breakfast Spot,Lounge,Skating Rink,Chinese Restaurant,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fried Chicken Joint,Coffee Shop,Sandwich Place,Discount Store,Pharmacy,Beer Store,Japanese Restaurant,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pharmacy,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Terminal,Dessert Shop


#### Cluster the neighborhoods into 5 clusters.

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0, 0, 1, 0, 3, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 4, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [51]:
toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Concert Hall,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1.0,Coffee Shop,Park,Café,Bakery,Pub,Theater,Breakfast Spot,Restaurant,Gym / Fitness Center,Mexican Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Sporting Goods Shop,Miscellaneous Shop,Athletics & Sports,Arts & Crafts Store,Boutique
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,1.0,Coffee Shop,Park,Gym,College Cafeteria,Smoothie Shop,Seafood Restaurant,Burger Joint,Burrito Place,Sandwich Place,Café


#### Visualize.

In [52]:
toronto_merged=toronto_merged.dropna()

In [53]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [54]:
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



#### Examine clusters.

#### Cluster 1

In [56]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Diner
10,North York,0,Japanese Restaurant,Park,Sushi Restaurant,Pub,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
21,York,0,Park,Fast Food Restaurant,Pharmacy,Market,Women's Store,Comic Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant
35,East York,0,Park,Pizza Place,Coffee Shop,Convenience Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
40,North York,0,Airport,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
46,North York,0,Grocery Store,Bank,Park,Shopping Mall,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
49,North York,0,Basketball Court,Bakery,Construction & Landscaping,Park,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
61,Central Toronto,0,Park,Bus Line,Swim School,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
64,York,0,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,0,Bank,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


#### Cluster 2

In [57]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Concert Hall,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,Downtown Toronto,1,Coffee Shop,Park,Café,Bakery,Pub,Theater,Breakfast Spot,Restaurant,Gym / Fitness Center,Mexican Restaurant
3,North York,1,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Sporting Goods Shop,Miscellaneous Shop,Athletics & Sports,Arts & Crafts Store,Boutique
4,Queen's Park,1,Coffee Shop,Park,Gym,College Cafeteria,Smoothie Shop,Seafood Restaurant,Burger Joint,Burrito Place,Sandwich Place,Café
6,Scarborough,1,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
7,North York,1,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Baseball Field,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
8,East York,1,Pizza Place,Fast Food Restaurant,Breakfast Spot,Gastropub,Gym / Fitness Center,Pharmacy,Intersection,Café,Athletics & Sports,Bank
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Diner,Pizza Place,Italian Restaurant,Japanese Restaurant
11,Etobicoke,1,Bank,Golf Course,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
12,Scarborough,1,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


#### Cluster 3

In [58]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2,Drugstore,Rental Car Location,Yoga Studio,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant


#### Cluster 4

In [59]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3,Food Truck,Baseball Field,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Diner
57,North York,3,Baseball Field,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
101,Etobicoke,3,Business Service,Pool,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 5

In [60]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,4,Playground,Convenience Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
83,Central Toronto,4,Playground,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
